# Movie Genre Predictor

In [4]:
import pandas as pd 
import numpy as np 
import kagglehub

In [ ]:
# Download latest version
path = kagglehub.dataset_download("guru001/movie-genre-prediction")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\shery\.cache\kagglehub\datasets\guru001\movie-genre-prediction\versions\1


In [ ]:
# Read the dataset, drop the id column 
df = pd.read_csv('./data/test.csv')
df.drop(columns="id")

,movie_name,synopsis,genre
0,A Death Sentence,"12 y.o. Ida's dad'll die without a DKK1,500,00...",action
1,Intermedio,A group of four teenage friends become trapped...,action
2,30 Chua Phai Tet,A guy left his home for 12 years till he came ...,action
3,Paranoiac,A man long believed dead returns to the family...,action
4,Ordinary Happiness,"After a deadly accident, Paolo comes back on E...",action
...,...,...,...
35995,Mysterious Doctor Satan,A mad scientist plots the domination of Americ...,action
35996,Blondes at Work,Steve promises his captain not to favor Torchy...,action
35997,The Hurricane,A Polynesian sailor unjustly imprisoned after ...,action
35998,Malayankunju,Anikkuttan is an ill-tempered electronics tech...,action


In [ ]:
# Remove punctuation 
def remove_punctuation(text): 
    punctuation =  ",./;'[]-=\\~!@#$%^&*()_+{}|\":?><"
    cleaned = "".join([c for c in text if c not in punctuation])
    return cleaned 

df["synopsis"] = df["synopsis"].apply(lambda x: remove_punctuation(x))
df.head(10)

,id,movie_name,synopsis,genre
0,16863,A Death Sentence,12 yo Idas dadll die without a DKK1500000 oper...,action
1,48456,Intermedio,A group of four teenage friends become trapped...,action
2,41383,30 Chua Phai Tet,A guy left his home for 12 years till he came ...,action
3,84007,Paranoiac,A man long believed dead returns to the family...,action
4,40269,Ordinary Happiness,After a deadly accident Paolo comes back on Ea...,action
5,16524,The Syndicate,An alcoholic his wife a down and out American ...,action
6,21245,12 Depths,Struck by amnesia and framed for murder a man ...,action
7,84681,The Feud,A mysterious man returns to his home in Iowa f...,action
8,73352,Cabin Fever: Patient Zero,When a group of friends enjoying a bachelor cr...,action
9,37966,Peregrina,After a near death experience a young woman be...,action
